# But du notebook

Ce notebook montre comment faire de la classification en sentiments en utilisant un modèle BERT et Hugging Face. On utilisera d'abord une pipeline déjà toute prête puis on fera la même chose mais en reconstituant les étapes une à une. 

# Importation des modules

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from torch.nn.functional import softmax as softmax

c:\Users\aengp\Desktop\memoire\.venvMemoire\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

# Utilisation d'une pipeline existante

In [4]:
classifier = pipeline("sentiment-analysis", model = checkpoint)
classifier(
    [
        "I really hate studying and reading, it's boring.",
        "I love japanese food",
    ]
)

[{'label': 'NEGATIVE', 'score': 0.9991845488548279},
 {'label': 'POSITIVE', 'score': 0.9995834231376648}]

# Sans pipeline

## Pre-processing

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
        "I really hate studying and reading, it's boring.",
        "I love japanese food",
]

inputs = tokenizer(raw_inputs, 
                   padding=True, # Ajout de 0 pour que toutes les phrases fassent la même taille
                   truncation=True, # Truncation des phrases qui dépassent le nombre maximum de caractères
                   return_tensors="pt") # Utilisation de tensors de PyTorch

inputs

{'input_ids': tensor([[  101,  1045,  2428,  5223,  5702,  1998,  3752,  1010,  2009,  1005,
          1055, 11771,  1012,   102],
        [  101,  1045,  2293,  2887,  2833,   102,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

## Model

In [6]:
# Chargement du checkpoint avec une architecture spécifique pour la classification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [8]:
outputs = model(**inputs)

## Post-processing

> The softmax function, also known as softargmax or normalized exponential function, converts a vector of $K$ real numbers into a probability distribution of $K$ possible outcomes. It is a generalization of the logistic function to multiple dimensions, and used in multinomial logistic regression. The softmax function is often used as the last activation function of a neural network to normalize the output of a network to a probability distribution over predicted output classes.

In [11]:
# Application d'une couche softmax pour obtenir des probabilités depuis les logits
predictions = softmax(outputs.logits, dim=-1)

predictions

tensor([[9.9918e-01, 8.1548e-04],
        [4.1661e-04, 9.9958e-01]], grad_fn=<SoftmaxBackward0>)

In [12]:
# Etiquettes correspondantes
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}